## Importing Test dataset

It's to check the models profeciency.


In [2]:
from google.colab import files
uploaded = files.upload()

Saving Person_flow.v3-01.yolov8.zip to Person_flow.v3-01.yolov8.zip


In [3]:
# Unzip
!unzip Person_flow.v3-01.yolov8.zip -d /content/Person_flow

Archive:  Person_flow.v3-01.yolov8.zip
  inflating: /content/Person_flow/README.dataset.txt  
  inflating: /content/Person_flow/README.roboflow.txt  
  inflating: /content/Person_flow/data.yaml  
   creating: /content/Person_flow/test/
   creating: /content/Person_flow/test/images/
 extracting: /content/Person_flow/test/images/people-walking_mp4-0004_jpg.rf.17f8359e4881b58572b8b0cf263d9c72.jpg  
 extracting: /content/Person_flow/test/images/people-walking_mp4-0014_jpg.rf.439c0b5422185facb211bac486624e7b.jpg  
 extracting: /content/Person_flow/test/images/people-walking_mp4-0036_jpg.rf.cfe545f62d34be5a64a5082892e50528.jpg  
 extracting: /content/Person_flow/test/images/people-walking_mp4-0037_jpg.rf.a2578ed4aa4265a70ee99b5bfb764802.jpg  
 extracting: /content/Person_flow/test/images/people-walking_mp4-0055_jpg.rf.3c73974a7a2886019c0ddc15e1e5c282.jpg  
 extracting: /content/Person_flow/test/images/people-walking_mp4-0057_jpg.rf.e497432aa97379dd816d2c919b24784d.jpg  
 extracting: /content

In [4]:
# Check for labels
!ls /content/Person_flow/test/labels

people-walking_mp4-0004_jpg.rf.17f8359e4881b58572b8b0cf263d9c72.txt
people-walking_mp4-0014_jpg.rf.439c0b5422185facb211bac486624e7b.txt
people-walking_mp4-0036_jpg.rf.cfe545f62d34be5a64a5082892e50528.txt
people-walking_mp4-0037_jpg.rf.a2578ed4aa4265a70ee99b5bfb764802.txt
people-walking_mp4-0055_jpg.rf.3c73974a7a2886019c0ddc15e1e5c282.txt
people-walking_mp4-0057_jpg.rf.e497432aa97379dd816d2c919b24784d.txt
people-walking_mp4-0070_jpg.rf.e9726c116acbbc062b92a82477074260.txt
people-walking_mp4-0086_jpg.rf.9c48e2a480e8c15b48d82ef7dbb1f7b9.txt
people-walking_mp4-0093_jpg.rf.79b36468d80eeea6c55be210546d52fa.txt
people-walking_mp4-0099_jpg.rf.83f473f73cd633e238ef6800d27d40ea.txt
people-walking_mp4-0106_jpg.rf.c9ae38cbbabf1926dfc525477e0bfc15.txt
people-walking_mp4-0110_jpg.rf.a6c39768155235a884ba9db459050e62.txt
people-walking_mp4-0127_jpg.rf.cce7cc436726ec7c18e1d31fb66a8c85.txt


# Install dependencies

In [5]:
# Install dependencies
!pip install -q ultralytics supervision opencv-python
!apt-get update -qq && apt-get install -y -qq ffmpeg

import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO
import os
from glob import glob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 20.5 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
# Download video
!wget -q https://media.roboflow.com/supervision/video-examples/people-walking.mp4

In [7]:
# Load video
VIDEO_PATH = "people-walking.mp4"
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
width, height = video_info.resolution_wh
fps = video_info.fps

## Define lines for IN/OUT counting

In [8]:
upper_y = int(height * 0.15)  # Upper line for IN (downward)
upper_start = sv.Point(0, upper_y)
upper_end = sv.Point(width, upper_y)
lower_y = int(height * 0.8)   # Lower line for OUT (upward)
lower_start = sv.Point(0, lower_y)
lower_end = sv.Point(width, lower_y)

upper_line = sv.LineZone(start=upper_start, end=upper_end)
lower_line = sv.LineZone(start=lower_start, end=lower_end)

In [9]:
# Initialize annotators
box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
upper_line_annotator = sv.LineZoneAnnotator(color=sv.Color.BLUE, display_in_count=True, custom_in_text="IN")
lower_line_annotator = sv.LineZoneAnnotator(color=sv.Color.RED, display_in_count=True, custom_in_text="OUT")
heat_map_annotator = sv.HeatMapAnnotator(position=sv.Position.CENTER, opacity=0.5, radius=20, kernel_size=25, top_hue=0, low_hue=125)

## Load model and tracker

In [10]:
model = YOLO("yolov8n.pt")
byte_tracker = sv.ByteTrack()

In [11]:
# Load Roboflow annotations
ANNOTATION_DIR = "/content/Person_flow/test/labels"
annotation_files = sorted(glob(os.path.join(ANNOTATION_DIR, "*.txt"))) if os.path.exists(ANNOTATION_DIR) else []
print(f"Using {len(annotation_files)} annotation files from {ANNOTATION_DIR}")

Using 13 annotation files from /content/Person_flow/test/labels


## Initialize VideoWriter

In [12]:
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter("output.mp4", fourcc, fps, (width, height))
if not writer.isOpened():
    fourcc = cv2.VideoWriter_fourcc(*"H264")
    writer = cv2.VideoWriter("output.mp4", fourcc, fps, (width, height))
    if not writer.isOpened():
        raise Exception("VideoWriter failed to initialize")
# Process video
generator = sv.get_video_frames_generator(VIDEO_PATH)
frame_count = 0
manual_in_count = 0
manual_out_count = 0
prev_annotated_centers = []

for frame in generator:
    frame_count += 1

    # Detect and track
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[detections.class_id == 0]  # Filter to people
    detections = byte_tracker.update_with_detections(detections)

    # Compute detected centers
    centers = [(int((xyxy[0] + xyxy[2]) / 2), int((xyxy[1] + xyxy[3]) / 2)) for xyxy in detections.xyxy]
    if centers:
        print(f"Frame {frame_count}: {len(detections)} detections, Centers: {centers}, Tracker IDs: {detections.tracker_id.tolist()}")

    # Load annotations
    annotated_centers = []
    if frame_count <= len(annotation_files):
        with open(annotation_files[frame_count - 1], 'r') as f:
            lines = f.readlines()
            for line in lines:
                parts = line.strip().split()
                if parts[0] == '0':  # Person class
                    x_center = float(parts[1]) * width
                    y_center = float(parts[2]) * height
                    annotated_centers.append((int(x_center), int(y_center)))
        print(f"Frame {frame_count}: Annotated Centers: {annotated_centers}")

        # Manual count from annotations
        for center in annotated_centers:
            y = center[1]
            if frame_count > 1:
                for prev_center in prev_annotated_centers:
                    prev_y = prev_center[1]
                    if prev_y < upper_y and y > upper_y:
                        manual_in_count += 1
                        print(f"Frame {frame_count}: Manual IN crossing detected")
                    if prev_y > lower_y and y < lower_y:
                        manual_out_count += 1
                        print(f"Frame {frame_count}: Manual OUT crossing detected")

    prev_annotated_centers = annotated_centers

    # Count crossings
    upper_crossings = upper_line.trigger(detections)
    lower_crossings = lower_line.trigger(detections)
    if upper_crossings or lower_crossings:
        print(f"Frame {frame_count}: Upper crossings: {upper_crossings}, Lower crossings: {lower_crossings}")
    print(f"Frame {frame_count}: IN count: {upper_line.in_count}, OUT count: {lower_line.in_count}")

    # Annotate frame
    annotated_frame = heat_map_annotator.annotate(frame.copy(), detections)
    for center in centers:
        cv2.circle(annotated_frame, center, 5, (0, 255, 0), -1)  # Green for detections
    for center in annotated_centers:
        cv2.circle(annotated_frame, center, 5, (0, 0, 255), -1)  # Red for annotations
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
    labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    annotated_frame = upper_line_annotator.annotate(annotated_frame, upper_line)
    annotated_frame = lower_line_annotator.annotate(annotated_frame, lower_line)

    # Write frame
    writer.write(annotated_frame)

writer.release()


Streaming output truncated to the last 5000 lines.
Frame 4: Manual OUT crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual IN crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manual OUT crossing detected
Frame 4: Manua

#heatmap

In [14]:
# Save
blank = np.zeros((height, width, 3), dtype=np.uint8)
heatmap_img = heat_map_annotator.annotate(blank, sv.Detections.empty())
cv2.imwrite("heatmap.png", heatmap_img)

print("Processing complete.")
print(f"Final Total IN: {upper_line.in_count}")
print(f"Final Total OUT: {lower_line.in_count}")
print(f"Manual IN from annotations: {manual_in_count}")
print(f"Manual OUT from annotations: {manual_out_count}")

# Download outputs
from google.colab import files
files.download('output.mp4')
files.download('heatmap.png')


Processing complete.
Final Total IN: 8
Final Total OUT: 7
Manual IN from annotations: 3298
Manual OUT from annotations: 1173


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The drive link for the Heatmap , Output video and Readme is [Here](https://drive.google.com/drive/folders/1pLS5XwHynmpa0ObYiF0pBcX7nvWVhODV?usp=sharing)